This code downloads financial reports data from the SDI API. Make sure to adjust years accordingly. 
Remember that FDIC limits downloads to 10,000 observations at a time, and years prior to 2000 contain > 10,000 observations. 
The code has been adjusted to account for years that may have more than 10,000 observations and downloads a separate file as "REPORTNAME_YYYYb.csv"

In [2]:
# Download Liabilities & Capital Data

import requests
import os
import time

def download_fdic_reports(start_year=1989, end_year=1999, output_dir="fdic_reports"):
    """
    Download FDIC financial reports for year-end dates from start_year to end_year.
    Handles pagination for years with more than 10,000 records.
    
    Args:
        start_year (int): First year to download (inclusive)
        end_year (int): Last year to download (inclusive)
        output_dir (str): Directory to save downloaded files
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Base URL for the FDIC API
    base_url = "https://banks.data.fdic.gov/api/financials"
    
    # Full list of fields from the provided URL
    fields = [
        "CERT", "RSSDHCR", "NAMEFULL", "CITY", "STALP", "ZIP", "REPDTE", "BKCLASS", 
        "NAMEHCR", "OFFDOM", "SPECGRP", "SUBCHAPS", "ESTYMD", "INSDATE", "EFFDATE", 
        "MUTUAL", "PARCERT", "TRUST", "REGAGNT", "INSAGNT1", "FDICDBS", "FDICSUPV", 
        "FLDOFF", "FED", "OCCDIST", "OTSREGNM", "OFFOA", "CB", "LIABEQ", "LIAB", 
        "DEP", "DEPDOM", "ESTINS", "TRN", "TRNIPCOC", "TRNUSGOV", "TRNMUNI", "TRNCBO", 
        "TRNFCFG", "NTR", "NTRIPC", "NTRUSGOV", "NTRMUNI", "NTRCOMOT", "NTRFCFG", 
        "DEPFOR", "DEPIPCCF", "DEPUSBKF", "DEPFBKF", "DEPFGOVF", "DEPUSMF", "DEPNIFOR", 
        "DEPIFOR", "DDT", "NTRSMMDA", "NTRSOTH", "TS", "DEPNIDOM", "DEPIDOM", "COREDEP", 
        "DEPINS", "DEPUNA", "IRAKEOGH", "BRO", "BROINS", "DEPLSNB", "DEPCSBQ", "DEPSMAMT", 
        "DEPSMB", "DEPLGAMT", "DEPLGB", "DEPSMRA", "DEPSMRN", "DEPLGRA", "DEPLGRN", 
        "TRNNIA", "TRNNIN", "NTRCDSM", "NTRTMMED", "NTRTMLGJ", "CD3LESS", "CD3T12S", 
        "CD1T3S", "CDOV3S", "CD3LES", "CD3T12", "CD1T3", "CDOV3", "FREPP", "TRADEL", 
        "OTHBRF", "OTBFH1L", "OTBFH1T3", "OTBFH3T5", "OTBFHOV5", "OTBFHSTA", "OTBOT1L", 
        "OTBOT1T3", "OTBOT3T5", "OTBOTOV5", "OTHBOT1L", "SUBND", "ALLOTHL", "EQTOT", 
        "EQ", "EQPP", "EQCS", "EQSUR", "EQUPTOT", "EQCONSUB", "EQCPREV", "EQCREST", 
        "NETINC", "EQCSTKRX", "EQCTRSTX", "EQCMRG", "EQCDIVP", "EQCDIVC", "EQCCOMPI", 
        "EQCBHCTR", "ASSTLT", "ASSET2", "ASSET5", "ERNAST", "OALIFINS", "OALIFGEN", 
        "OALIFHYB", "OALIFSEP", "AVASSETJ", "RWAJT", "RBCT2", "RBCT1J", "OTHBFHLB", "VOLIAB"
    ]
    
    # Define limits and offsets for pagination
    limit = 10000
    offsets = [0, 10000]  # First batch: offset 0, second batch: offset 10000
    
    # Iterate through each year
    for year in range(start_year, end_year + 1):
        # Format the date as YYYYMMDD (December 31st)
        report_date = f"{year}1231"
        
        # Construct the base filter
        filters = f"ACTIVE:1 AND !(BKCLASS:NC) AND REPDTE:{report_date}"
        
        # Download data with pagination
        for i, offset in enumerate(offsets):
            # Create appropriate suffix for the filename
            suffix = "" if i == 0 else "b"
            filename = f"liabilities_{year}{suffix}.csv"
            file_path = os.path.join(output_dir, filename)
            
            print(f"Downloading FDIC report for year {year} (batch {i+1})...")
            
            # Construct parameters with the current offset
            params = {
                "filters": filters,
                "fields": ",".join(fields),
                "sort_by": "REPDTE",
                "sort_order": "DESC",
                "limit": str(limit),
                "offset": str(offset),
                "format": "csv",
                "download": "true",
                "filename": f"liabilities_{year}{suffix}"
            }
            
            try:
                # Send the request
                response = requests.get(base_url, params=params)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Check the content before saving
                    content = response.content
                    content_str = content.decode('utf-8', errors='replace')
                    
                    # Basic validation - check if we got more than just a header row
                    num_lines = content_str.count('\n')
                    if num_lines < 2:
                        print(f"Warning: Downloaded file for {year} (batch {i+1}) only has {num_lines+1} lines")
                        
                        # If it's the second batch and there's only a header, it means there are no more records
                        if i > 0 and num_lines <= 1:
                            print(f"No more records for year {year} after offset {offset}. Skipping this file.")
                            continue
                        
                        # If there's an error message in the response, print it
                        if "error" in content_str.lower() or "invalid" in content_str.lower():
                            print(f"Error in response: {content_str}")
                            continue
                    
                    # Save the content to a file
                    with open(file_path, 'wb') as f:
                        f.write(content)
                    
                    print(f"Successfully downloaded {filename} ({num_lines+1} lines)")
                    
                    # If there are fewer than limit records in the first batch, we don't need the second batch
                    if i == 0 and num_lines < limit:
                        print(f"Year {year} has fewer than {limit} records. Skipping second batch.")
                        break
                        
                else:
                    print(f"Failed to download report for {year} (batch {i+1}). Status code: {response.status_code}")
                    if hasattr(response, 'text'):
                        print(f"Error details: {response.text[:200]}...")
            
            except Exception as e:
                print(f"Error downloading report for {year} (batch {i+1}): {str(e)}")
                
            # Add a delay between requests to avoid overwhelming the server
            time.sleep(2)
    
    print(f"\nDownload complete. Files saved to '{output_dir}' directory.")

if __name__ == "__main__":
    # Execute the download function for years 1989-1999
    download_fdic_reports(1989, 1999, "C:/temp/fdic_reports")


Successfully downloaded liabilities_1989.csv (10001 lines)
Successfully downloaded liabilities_1989b.csv (6109 lines)
Successfully downloaded liabilities_1990.csv (10001 lines)
Successfully downloaded liabilities_1990b.csv (5375 lines)
Successfully downloaded liabilities_1991.csv (10001 lines)
Successfully downloaded liabilities_1991b.csv (4613 lines)
Successfully downloaded liabilities_1992.csv (10001 lines)
Successfully downloaded liabilities_1992b.csv (3974 lines)
Successfully downloaded liabilities_1993.csv (10001 lines)
Successfully downloaded liabilities_1993b.csv (3326 lines)
Successfully downloaded liabilities_1994.csv (10001 lines)
Successfully downloaded liabilities_1994b.csv (2645 lines)
Successfully downloaded liabilities_1995.csv (10001 lines)
Successfully downloaded liabilities_1995b.csv (2004 lines)
Successfully downloaded liabilities_1996.csv (10001 lines)
Successfully downloaded liabilities_1996b.csv (1481 lines)
Successfully downloaded liabilities_1997.csv (10001 line

In [4]:
# Download Past Due & Non-Accrual Data
import requests
import os
import time
import csv
import io

def download_fdic_reports(start_year=1989, end_year=2023, output_dir="fdic_reports"):
    """
    Download FDIC financial reports for year-end dates from start_year to end_year.
    Handles pagination for years with more than 10,000 records.
    
    Args:
        start_year (int): First year to download (inclusive)
        end_year (int): Last year to download (inclusive)
        output_dir (str): Directory to save downloaded files
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Base URL for the FDIC API
    base_url = "https://banks.data.fdic.gov/api/financials"
    
    # Full list of fields from the provided URL - these are non-performing asset fields
    fields = [
        "CERT", "RSSDHCR", "NAMEFULL", "CITY", "STALP", "ZIP", "REPDTE", "BKCLASS", 
        "NAMEHCR", "OFFDOM", "SPECGRP", "SUBCHAPS", "ESTYMD", "INSDATE", "EFFDATE", 
        "MUTUAL", "PARCERT", "TRUST", "REGAGNT", "INSAGNT1", "FDICDBS", "FDICSUPV", 
        "FLDOFF", "FED", "OCCDIST", "OTSREGNM", "OFFOA", "CB", "P3ASSET", "P3RE", 
        "P3RENUS", "P3RELNDO", "P3RECONS", "P3RECNFM", "P3RECNOT", "P3REAG", "P3RERES", 
        "P3RERSFM", "P3RERSF2", "P3RELOC", "P3REMULT", "P3RENRES", "P3RENROW", "P3RENROT", 
        "P3REFOR", "P3DEP", "P3DEPNUS", "P3AG", "P3CI", "P3CINUS", "P3CON", "P3CRCD", 
        "P3AUTO", "P3CONOTH", "P3FG", "P3OTHLN", "P3LS", "P3SCDEBT", "P3AGSM", "P3LNSALE", 
        "P3GTYPAR", "P3GTY", "P3GTYGNM", "P9ASSET", "P9RE", "P9RENUS", "P9RELNDO", 
        "P9RECONS", "P9RECNFM", "P9RECNOT", "P9REAG", "P9RERES", "P9RERSFM", "P9RERSF2", 
        "P9RELOC", "P9REMULT", "P9RENRES", "P9RENROW", "P9RENROT", "P9REFOR", "P9DEP", 
        "P9DEPNUS", "P9AG", "P9CI", "P9CINUS", "P9CON", "P9CRCD", "P9AUTO", "P9CONOTH", 
        "P9FG", "P9OTHLN", "P9LS", "P9SCDEBT", "P9AGSM", "P9LNSALE", "P9GTYPAR", "P9GTY", 
        "P9GTYGNM", "NAASSET", "NARE", "NARENUS", "NARELNDO", "NARECONS", "NARECNFM", 
        "NARECNOT", "NAREAG", "NARERES", "NARERSFM", "NARERSF2", "NARELOC", "NAREMULT", 
        "NARENRES", "NARENROW", "NARENROT", "NAREFOR", "NADEP", "NADEPNUS", "NAAG", 
        "NACI", "NACINUS", "NACON", "NACRCD", "NAAUTO", "NACONOTH", "NAFG", "NAOTHLN", 
        "NALS", "NASCDEBT", "NAAGSM", "NALNSALE", "NAGTYPAR", "NAGTY", "NAGTYGNM"
    ]
    
    # Define limits and offsets for pagination
    limit = 10000
    offsets = [0, 10000]  # First batch: offset 0, second batch: offset 10000
    
    # Use year-end dates (December 31) for each year
    for year in range(start_year, end_year + 1):
        # Format the date as YYYYMMDD (December 31st)
        report_date = f"{year}1231"
        
        # Construct the base filter
        filters = f"ACTIVE:1 AND !(BKCLASS:NC) AND REPDTE:{report_date}"
        
        # Download data with pagination
        for i, offset in enumerate(offsets):
            # Create appropriate suffix for the filename - changed to use "nonperf" prefix
            suffix = "" if i == 0 else "b"
            filename = f"nonperf_{year}{suffix}.csv"
            file_path = os.path.join(output_dir, filename)
            
            print(f"Downloading FDIC non-performing assets report for year {year} (batch {i+1})...")
            
            # Construct parameters with the current offset
            params = {
                "filters": filters,
                "fields": ",".join(fields),
                "sort_by": "REPDTE",
                "sort_order": "DESC",
                "limit": str(limit),
                "offset": str(offset),
                "format": "csv",
                "download": "true",
                "filename": f"nonperf_{year}{suffix}"
            }
            
            try:
                # Send the request
                response = requests.get(base_url, params=params)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Check the content before saving
                    content = response.content
                    content_str = content.decode('utf-8', errors='replace')
                    
                    # Check if we have valid CSV data with at least a header row
                    csv_reader = csv.reader(io.StringIO(content_str))
                    rows = list(csv_reader)
                    num_rows = len(rows)
                    
                    if num_rows <= 1:
                        print(f"Warning: Downloaded file for {year} (batch {i+1}) only has {num_rows} rows")
                        
                        # If it's the second batch and there's only a header or no data, it means there are no more records
                        if i > 0 and num_rows <= 1:
                            print(f"No more records for year {year} after offset {offset}. Skipping this file.")
                            continue
                        
                        # If there's an error message in the response, print it
                        if "error" in content_str.lower() or "invalid" in content_str.lower():
                            print(f"Error in response: {content_str[:200]}...")
                            continue
                    
                    # Save the content to a file
                    with open(file_path, 'wb') as f:
                        f.write(content)
                    
                    print(f"Successfully downloaded {filename} ({num_rows} rows)")
                    
                    # If there are fewer than limit records in the first batch, we don't need the second batch
                    if i == 0 and num_rows < limit + 1:  # +1 for header row
                        print(f"Year {year} has fewer than {limit} records. Skipping second batch.")
                        break
                        
                else:
                    print(f"Failed to download report for {year} (batch {i+1}). Status code: {response.status_code}")
                    if hasattr(response, 'text'):
                        print(f"Error details: {response.text[:200]}...")
            
            except Exception as e:
                print(f"Error downloading report for {year} (batch {i+1}): {str(e)}")
                
            # Add a delay between requests to avoid overwhelming the server
            time.sleep(3)  # Increased delay to 3 seconds to be more respectful to the server
    
    print(f"\nDownload complete. Files saved to '{output_dir}' directory.")

if __name__ == "__main__":
    # Execute the download function for years 1989-2023
    download_fdic_reports(1989, 2023, "C:/temp/fdic_reports")

Successfully downloaded nonperf_1989.csv (10001 rows)
Successfully downloaded nonperf_1989b.csv (6109 rows)
Successfully downloaded nonperf_1990.csv (10001 rows)
Successfully downloaded nonperf_1990b.csv (5375 rows)
Successfully downloaded nonperf_1991.csv (10001 rows)
Successfully downloaded nonperf_1991b.csv (4613 rows)
Successfully downloaded nonperf_1992.csv (10001 rows)
Successfully downloaded nonperf_1992b.csv (3974 rows)
Successfully downloaded nonperf_1993.csv (10001 rows)
Successfully downloaded nonperf_1993b.csv (3326 rows)
Successfully downloaded nonperf_1994.csv (10001 rows)
Successfully downloaded nonperf_1994b.csv (2645 rows)
Successfully downloaded nonperf_1995.csv (10001 rows)
Successfully downloaded nonperf_1995b.csv (2004 rows)
Successfully downloaded nonperf_1996.csv (10001 rows)
Successfully downloaded nonperf_1996b.csv (1481 rows)
Successfully downloaded nonperf_1997.csv (10001 rows)
Successfully downloaded nonperf_1997b.csv (947 rows)
Successfully downloaded nonpe

In [6]:
# Download Assets Data
import requests
import os
import time
import csv
import io

def download_fdic_reports(start_year=1989, end_year=2023, output_dir="fdic_reports"):
    """
    Download FDIC financial reports (asset data) for year-end dates from start_year to end_year.
    Handles pagination for years with more than 10,000 records.
    
    Args:
        start_year (int): First year to download (inclusive)
        end_year (int): Last year to download (inclusive)
        output_dir (str): Directory to save downloaded files
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Base URL for the FDIC API
    base_url = "https://banks.data.fdic.gov/api/financials"
    
    # Full list of fields from the provided URL - these are asset-related fields
    fields = [
        "CERT", "RSSDHCR", "NAMEFULL", "CITY", "STALP", "ZIP", "REPDTE", "BKCLASS", 
        "NAMEHCR", "OFFDOM", "SPECGRP", "SUBCHAPS", "ESTYMD", "INSDATE", "EFFDATE", 
        "MUTUAL", "PARCERT", "TRUST", "REGAGNT", "INSAGNT1", "FDICDBS", "FDICSUPV", 
        "FLDOFF", "FED", "OCCDIST", "OTSREGNM", "OFFOA", "CB", "ASSET", "CHBAL", 
        "CHBALNI", "CHBALI", "CHCIC", "CHITEM", "CHCOIN", "CHUS", "CHNUS", "CHFRB", 
        "SC", "SCUS", "SCUST", "SCUSO", "SCASPNSUM", "SCFMN", "SCGNM", "SCCOL", 
        "SCCPTG", "SCCMOG", "SCMUNI", "SCDOMO", "SCRMBPI", "SCCMOS", "SCABS", "SCSFP", 
        "SCODOT", "SCFORD", "SCEQNFT", "SCEQ", "SCEQFV", "SCHTMRES", "SCTATFR", 
        "SCPLEDGE", "SCMTGBK", "SCGTY", "SCODPC", "SCODPI", "SCCMPT", "SCCMOT", "SCHA", 
        "SCAF", "SCRDEBT", "SCPT3LES", "SCPT3T12", "SCPT3T5", "SCPT5T15", "SCPTOV15", 
        "SCO3YLES", "SCOOV3Y", "SCNM3LES", "SCNM3T12", "SCNM1T3", "SCNM3T5", "SCNM5T15", 
        "SCNMOV15", "SC1LES", "SCSNHAA", "SCSNHAF", "TRADE", "TRREVALSUM", "TRLREVAL", 
        "FREPO", "LNLSNET", "LNATRES", "LNLSGR", "LNCONTRA", "LNLSGRS", "LNRE", 
        "LNREDOM", "LNRECONS", "LNRENRES", "LNREMULT", "LNRERES", "LNREAG", "LNREFOR", 
        "LNAG", "LNCI", "LNCON", "LNCRCD", "LNCONRP", "LNAUTO", "LNCONOTH", "LNOTCI", 
        "LNFG", "LNMUNI", "LNSOTHER", "LS", "LNCOMRE", "LNRENUS", "LNPLEDGE", 
        "RB2LNRES", "LNLSSALE", "LNEXAMT", "LNRENROW", "LNRENROT", "LNRERSFM", 
        "LNRERSF2", "LNRELOC", "LNRERSF1", "LNRECNFM", "LNRECNOT", "RSLNLTOT", 
        "RSLNREFM", "RSLNLS", "P3RSLNLT", "P3RSLNFM", "P3RSLNLS", "P9RSLNLT", 
        "P9RSLNFM", "P9RSLNLS", "NARSLNLT", "NARSLNFM", "NARSLNLS", "LNLSGRF", 
        "UNINCFOR", "LNLSFOR", "LNDEPAOBK", "LNDEPCBF", "LNDEPUSF", "LNDEPFCF", 
        "LNAGFOR", "LNCIFOR", "LNCINUSF", "LNCONFOR", "LNFGFOR", "LNMUNIF", "LNOTHERF", 
        "LSFOR", "LNRS3LES", "LNRS3T12", "LNRS1T3", "LNRS3T5", "LNRS5T15", "LNRSOV15", 
        "LNOT3LES", "LNOT3T12", "LNOT1T3", "LNOT3T5", "LNOT5T15", "LNOTOV15", "LNRENR4", 
        "LNRENR1", "LNRENR2", "LNRENR3", "LNCI4", "LNCI1", "LNCI2", "LNCI3", "LNREAG4", 
        "LNREAG1", "LNREAG2", "LNREAG3", "LNAG4", "LNAG1", "LNAG2", "LNAG3", "LNRENR4N", 
        "LNRENR1N", "LNRENR2N", "LNRENR3N", "LNCI4N", "LNCI1N", "LNCI2N", "LNCI3N", 
        "LNREAG4N", "LNREAG1N", "LNREAG2N", "LNREAG3N", "LNAG4N", "LNAG1N", "LNAG2N", 
        "LNAG3N", "PPPLNNUM", "PPPLNBAL", "PPPLNPLG", "PPPLF1LS", "PPPLFOV1", "AVPPPPLG", 
        "MMLFBAL", "AVMMLF", "BKPREM", "ORE", "OREINV", "OREOTH", "ORERES", "OREMULT", 
        "ORENRES", "ORECONS", "OREAG", "OREOTHF", "INTAN", "INTANGW", "INTANMSR", 
        "INTANOTH", "AOA"
    ]
    
    # Define limits and offsets for pagination
    limit = 10000
    offsets = [0, 10000]  # First batch: offset 0, second batch: offset 10000
    
    # Use year-end dates (December 31) for each year
    for year in range(start_year, end_year + 1):
        # Format the date as YYYYMMDD (December 31st)
        report_date = f"{year}1231"
        
        # Construct the base filter
        filters = f"ACTIVE:1 AND !(BKCLASS:NC) AND REPDTE:{report_date}"
        
        # Download data with pagination
        for i, offset in enumerate(offsets):
            # Create appropriate suffix for the filename - changed to use "assets" prefix
            suffix = "" if i == 0 else "b"
            filename = f"assets_{year}{suffix}.csv"
            file_path = os.path.join(output_dir, filename)
            
            print(f"Downloading FDIC asset report for year {year} (batch {i+1})...")
            
            # Construct parameters with the current offset
            params = {
                "filters": filters,
                "fields": ",".join(fields),
                "sort_by": "REPDTE",
                "sort_order": "DESC",
                "limit": str(limit),
                "offset": str(offset),
                "format": "csv",
                "download": "true",
                "filename": f"assets_{year}{suffix}"
            }
            
            try:
                # Send the request
                response = requests.get(base_url, params=params)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Check the content before saving
                    content = response.content
                    content_str = content.decode('utf-8', errors='replace')
                    
                    # Check if we have valid CSV data with at least a header row
                    csv_reader = csv.reader(io.StringIO(content_str))
                    rows = list(csv_reader)
                    num_rows = len(rows)
                    
                    if num_rows <= 1:
                        print(f"Warning: Downloaded file for {year} (batch {i+1}) only has {num_rows} rows")
                        
                        # If it's the second batch and there's only a header or no data, it means there are no more records
                        if i > 0 and num_rows <= 1:
                            print(f"No more records for year {year} after offset {offset}. Skipping this file.")
                            continue
                        
                        # If there's an error message in the response, print it
                        if "error" in content_str.lower() or "invalid" in content_str.lower():
                            print(f"Error in response: {content_str[:200]}...")
                            continue
                    
                    # Save the content to a file
                    with open(file_path, 'wb') as f:
                        f.write(content)
                    
                    print(f"Successfully downloaded {filename} ({num_rows} rows)")
                    
                    # If there are fewer than limit records in the first batch, we don't need the second batch
                    if i == 0 and num_rows < limit + 1:  # +1 for header row
                        print(f"Year {year} has fewer than {limit} records. Skipping second batch.")
                        break
                        
                else:
                    print(f"Failed to download report for {year} (batch {i+1}). Status code: {response.status_code}")
                    if hasattr(response, 'text'):
                        print(f"Error details: {response.text[:200]}...")
            
            except Exception as e:
                print(f"Error downloading report for {year} (batch {i+1}): {str(e)}")
                
            # Add a delay between requests to avoid overwhelming the server
            time.sleep(3)  # Increased delay to 3 seconds to be more respectful to the server
    
    print(f"\nDownload complete. Files saved to '{output_dir}' directory.")

if __name__ == "__main__":
    # Execute the download function for years 1989-2023
    download_fdic_reports(1989, 2023, "C:/temp/fdic_reports")

Successfully downloaded assets_1989.csv (10001 rows)
Successfully downloaded assets_1989b.csv (6109 rows)
Successfully downloaded assets_1990.csv (10001 rows)
Successfully downloaded assets_1990b.csv (5375 rows)
Successfully downloaded assets_1991.csv (10001 rows)
Successfully downloaded assets_1991b.csv (4613 rows)
Successfully downloaded assets_1992.csv (10001 rows)
Successfully downloaded assets_1992b.csv (3974 rows)
Successfully downloaded assets_1993.csv (10001 rows)
Successfully downloaded assets_1993b.csv (3326 rows)
Successfully downloaded assets_1994.csv (10001 rows)
Successfully downloaded assets_1994b.csv (2645 rows)
Successfully downloaded assets_1995.csv (10001 rows)
Successfully downloaded assets_1995b.csv (2004 rows)
Successfully downloaded assets_1996.csv (10001 rows)
Successfully downloaded assets_1996b.csv (1481 rows)
Successfully downloaded assets_1997.csv (10001 rows)
Successfully downloaded assets_1997b.csv (947 rows)
Successfully downloaded assets_1998.csv (10001 

In [8]:
# Down load Charged-Off Loan Data
import requests
import os
import time
import csv
import io

def download_fdic_reports(start_year=1989, end_year=2023, output_dir="fdic_reports"):
    """
    Download FDIC financial reports (charge-off data) for year-end dates from start_year to end_year.
    Handles pagination for years with more than 10,000 records.
    
    Args:
        start_year (int): First year to download (inclusive)
        end_year (int): Last year to download (inclusive)
        output_dir (str): Directory to save downloaded files
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Base URL for the FDIC API
    base_url = "https://banks.data.fdic.gov/api/financials"
    
    # Full list of fields from the provided URL - these are charge-off related fields
    fields = [
        "CERT", "RSSDHCR", "NAMEFULL", "CITY", "STALP", "ZIP", "REPDTE", "BKCLASS", 
        "NAMEHCR", "OFFDOM", "SPECGRP", "SUBCHAPS", "ESTYMD", "INSDATE", "EFFDATE", 
        "MUTUAL", "PARCERT", "TRUST", "REGAGNT", "INSAGNT1", "FDICDBS", "FDICSUPV", 
        "FLDOFF", "FED", "OCCDIST", "OTSREGNM", "OFFOA", "CB", "NTLNLS", "NTRE", 
        "NTRENUS", "NTREOFFDOM", "NTRECONS", "NTREAG", "NTRERES", "NTRERSFM", 
        "NTRERSF2", "NTRELOC", "NTREMULT", "NTRENRES", "NTREFOR", "NTDEP", "NTDEPNUS", 
        "NTAG", "NTCI", "NTCINUS", "NTCON", "NTCRCD", "NTAUTO", "NTCONOTH", "NTFORGV", 
        "NTOTHER", "NTLS", "NTAGSM", "DRLNLS", "DRRE", "DRRENUS", "DRREOFFDOM", 
        "DRRECONS", "DRREAG", "DRRERES", "DRRERSFM", "DRRERSF2", "DRRELOC", "DRREMULT", 
        "DRRENRES", "DRREFOR", "DRDEP", "DRDEPNUS", "DRAG", "DRCI", "DRCINUS", "DRCON", 
        "DRCRCD", "DRAUTO", "DRCONOTH", "DRFORGV", "DROTHER", "DRLS", "DRAGSM", 
        "CRLNLS", "CRRE", "CRRENUS", "CRREOFFDOM", "CRRECONS", "CRREAG", "CRRERES", 
        "CRRERSFM", "CRRERSF2", "CRRELOC", "CRREMULT", "CRRENRES", "CRREFOR", "CRDEP", 
        "CRDEPNUS", "CRAG", "CRCI", "CRCINUS", "CRCON", "CRCRCD", "CRAUTO", "CRCONOTH", 
        "CRFORGV", "CROTHER", "CRLS", "CRAGSM"
    ]
    
    # Define limits and offsets for pagination
    limit = 10000
    offsets = [0, 10000]  # First batch: offset 0, second batch: offset 10000
    
    # Use year-end dates (December 31) for each year
    for year in range(start_year, end_year + 1):
        # Format the date as YYYYMMDD (December 31st)
        report_date = f"{year}1231"
        
        # Construct the base filter
        filters = f"ACTIVE:1 AND !(BKCLASS:NC) AND REPDTE:{report_date}"
        
        # Download data with pagination
        for i, offset in enumerate(offsets):
            # Create appropriate suffix for the filename - changed to use "chgoffs" prefix
            suffix = "" if i == 0 else "b"
            filename = f"chgoffs_{year}{suffix}.csv"
            file_path = os.path.join(output_dir, filename)
            
            print(f"Downloading FDIC charge-off report for year {year} (batch {i+1})...")
            
            # Construct parameters with the current offset
            params = {
                "filters": filters,
                "fields": ",".join(fields),
                "sort_by": "REPDTE",
                "sort_order": "DESC",
                "limit": str(limit),
                "offset": str(offset),
                "format": "csv",
                "download": "true",
                "filename": f"chgoffs_{year}{suffix}"
            }
            
            try:
                # Send the request
                response = requests.get(base_url, params=params)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Check the content before saving
                    content = response.content
                    content_str = content.decode('utf-8', errors='replace')
                    
                    # Check if we have valid CSV data with at least a header row
                    csv_reader = csv.reader(io.StringIO(content_str))
                    rows = list(csv_reader)
                    num_rows = len(rows)
                    
                    if num_rows <= 1:
                        print(f"Warning: Downloaded file for {year} (batch {i+1}) only has {num_rows} rows")
                        
                        # If it's the second batch and there's only a header or no data, it means there are no more records
                        if i > 0 and num_rows <= 1:
                            print(f"No more records for year {year} after offset {offset}. Skipping this file.")
                            continue
                        
                        # If there's an error message in the response, print it
                        if "error" in content_str.lower() or "invalid" in content_str.lower():
                            print(f"Error in response: {content_str[:200]}...")
                            continue
                    
                    # Save the content to a file
                    with open(file_path, 'wb') as f:
                        f.write(content)
                    
                    print(f"Successfully downloaded {filename} ({num_rows} rows)")
                    
                    # If there are fewer than limit records in the first batch, we don't need the second batch
                    if i == 0 and num_rows < limit + 1:  # +1 for header row
                        print(f"Year {year} has fewer than {limit} records. Skipping second batch.")
                        break
                        
                else:
                    print(f"Failed to download report for {year} (batch {i+1}). Status code: {response.status_code}")
                    if hasattr(response, 'text'):
                        print(f"Error details: {response.text[:200]}...")
            
            except Exception as e:
                print(f"Error downloading report for {year} (batch {i+1}): {str(e)}")
                
            # Add a delay between requests to avoid overwhelming the server
            time.sleep(3)  # Increased delay to 3 seconds to be more respectful to the server
    
    print(f"\nDownload complete. Files saved to '{output_dir}' directory.")

if __name__ == "__main__":
    # Execute the download function for years 1989-2023
    download_fdic_reports(1989, 2023, "C:/temp/fdic_reports")

Successfully downloaded chgoffs_1989.csv (10001 rows)
Successfully downloaded chgoffs_1989b.csv (6109 rows)
Successfully downloaded chgoffs_1990.csv (10001 rows)
Successfully downloaded chgoffs_1990b.csv (5375 rows)
Successfully downloaded chgoffs_1991.csv (10001 rows)
Successfully downloaded chgoffs_1991b.csv (4613 rows)
Successfully downloaded chgoffs_1992.csv (10001 rows)
Successfully downloaded chgoffs_1992b.csv (3974 rows)
Successfully downloaded chgoffs_1993.csv (10001 rows)
Successfully downloaded chgoffs_1993b.csv (3326 rows)
Successfully downloaded chgoffs_1994.csv (10001 rows)
Successfully downloaded chgoffs_1994b.csv (2645 rows)
Successfully downloaded chgoffs_1995.csv (10001 rows)
Successfully downloaded chgoffs_1995b.csv (2004 rows)
Successfully downloaded chgoffs_1996.csv (10001 rows)
Successfully downloaded chgoffs_1996b.csv (1481 rows)
Successfully downloaded chgoffs_1997.csv (10001 rows)
Successfully downloaded chgoffs_1997b.csv (947 rows)
Successfully downloaded chgof

In [10]:
# Download Ratio Data
import requests
import os
import time
import csv
import io

def download_fdic_reports(start_year=1989, end_year=2023, output_dir="fdic_reports"):
    """
    Download FDIC financial reports (financial ratios data) for year-end dates from start_year to end_year.
    Handles pagination for years with more than 10,000 records.
    
    Args:
        start_year (int): First year to download (inclusive)
        end_year (int): Last year to download (inclusive)
        output_dir (str): Directory to save downloaded files
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Base URL for the FDIC API
    base_url = "https://banks.data.fdic.gov/api/financials"
    
    # Full list of fields from the provided URL - these are financial ratios fields
    fields = [
        "CERT", "RSSDHCR", "NAMEFULL", "CITY", "STALP", "ZIP", "REPDTE", "BKCLASS", 
        "NAMEHCR", "OFFDOM", "SPECGRP", "SUBCHAPS", "ESTYMD", "INSDATE", "EFFDATE", 
        "MUTUAL", "PARCERT", "TRUST", "REGAGNT", "INSAGNT1", "FDICDBS", "FDICSUPV", 
        "FLDOFF", "FED", "OCCDIST", "OTSREGNM", "OFFOA", "CB", "NTINCL", "NTINCHPP", 
        "INTINCY", "INTEXPY", "NIMY", "NONIIAY", "NONIXAY", "ELNATRY", "NOIJY", "ROA", 
        "ROAPTX", "ROE", "ROEINJR", "NTLNLSR", "NTRER", "NTRECOSR", "NTRENRSR", 
        "NTREMULR", "NTRERESR", "NTRELOCR", "NTREOTHR", "IDNTCIR", "IDNTCONR", 
        "IDNTCRDR", "IDNTCOOR", "NTAUTOPR", "NTCONOTR", "NTALLOTHR", "NTCOMRER", 
        "ELNANTR", "IDERNCVR", "EEFFR", "ASTEMPM", "EQCDIVNTINC", "ERNASTR", "LNATRESR", 
        "LNRESNCR", "NPERFV", "NCLNLS", "NCLNLSR", "NCRER", "NCRECONR", "NCRENRER", 
        "NCREMULR", "NCRERESR", "NCRELOCR", "NCREREOR", "IDNCCIR", "IDNCCONR", 
        "IDNCCRDR", "IDNCCOOR", "IDNCATOR", "IDNCCOTR", "IDNCOTHR", "NCCOMRER", 
        "IDNCGTPR", "LNLSNTV", "LNLSDEPR", "IDLNCORR", "DEPDASTR", "EQV", "RBC1AAJ", 
        "CBLRIND", "IDT1CER", "IDT1RWAJR", "RBCRWAJ"
    ]
    
    # Define limits and offsets for pagination
    limit = 10000
    offsets = [0, 10000]  # First batch: offset 0, second batch: offset 10000
    
    # Use year-end dates (December 31) for each year
    for year in range(start_year, end_year + 1):
        # Format the date as YYYYMMDD (December 31st)
        report_date = f"{year}1231"
        
        # Construct the base filter
        filters = f"ACTIVE:1 AND !(BKCLASS:NC) AND REPDTE:{report_date}"
        
        # Download data with pagination
        for i, offset in enumerate(offsets):
            # Create appropriate suffix for the filename - changed to use "ratios" prefix
            suffix = "" if i == 0 else "b"
            filename = f"ratios_{year}{suffix}.csv"
            file_path = os.path.join(output_dir, filename)
            
            print(f"Downloading FDIC financial ratios report for year {year} (batch {i+1})...")
            
            # Construct parameters with the current offset
            params = {
                "filters": filters,
                "fields": ",".join(fields),
                "sort_by": "REPDTE",
                "sort_order": "DESC",
                "limit": str(limit),
                "offset": str(offset),
                "format": "csv",
                "download": "true",
                "filename": f"ratios_{year}{suffix}"
            }
            
            try:
                # Send the request
                response = requests.get(base_url, params=params)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Check the content before saving
                    content = response.content
                    content_str = content.decode('utf-8', errors='replace')
                    
                    # Check if we have valid CSV data with at least a header row
                    csv_reader = csv.reader(io.StringIO(content_str))
                    rows = list(csv_reader)
                    num_rows = len(rows)
                    
                    if num_rows <= 1:
                        print(f"Warning: Downloaded file for {year} (batch {i+1}) only has {num_rows} rows")
                        
                        # If it's the second batch and there's only a header or no data, it means there are no more records
                        if i > 0 and num_rows <= 1:
                            print(f"No more records for year {year} after offset {offset}. Skipping this file.")
                            continue
                        
                        # If there's an error message in the response, print it
                        if "error" in content_str.lower() or "invalid" in content_str.lower():
                            print(f"Error in response: {content_str[:200]}...")
                            continue
                    
                    # Save the content to a file
                    with open(file_path, 'wb') as f:
                        f.write(content)
                    
                    print(f"Successfully downloaded {filename} ({num_rows} rows)")
                    
                    # If there are fewer than limit records in the first batch, we don't need the second batch
                    if i == 0 and num_rows < limit + 1:  # +1 for header row
                        print(f"Year {year} has fewer than {limit} records. Skipping second batch.")
                        break
                        
                else:
                    print(f"Failed to download report for {year} (batch {i+1}). Status code: {response.status_code}")
                    if hasattr(response, 'text'):
                        print(f"Error details: {response.text[:200]}...")
            
            except Exception as e:
                print(f"Error downloading report for {year} (batch {i+1}): {str(e)}")
                
            # Add a delay between requests to avoid overwhelming the server
            time.sleep(3)  # Increased delay to 3 seconds to be more respectful to the server
    
    print(f"\nDownload complete. Files saved to '{output_dir}' directory.")

if __name__ == "__main__":
    # Execute the download function for years 1989-2023
    download_fdic_reports(1989, 2023, "C:/temp/fdic_reports")

Successfully downloaded ratios_1989.csv (10001 rows)
Successfully downloaded ratios_1989b.csv (6109 rows)
Successfully downloaded ratios_1990.csv (10001 rows)
Successfully downloaded ratios_1990b.csv (5375 rows)
Successfully downloaded ratios_1991.csv (10001 rows)
Successfully downloaded ratios_1991b.csv (4613 rows)
Successfully downloaded ratios_1992.csv (10001 rows)
Successfully downloaded ratios_1992b.csv (3974 rows)
Successfully downloaded ratios_1993.csv (10001 rows)
Successfully downloaded ratios_1993b.csv (3326 rows)
Successfully downloaded ratios_1994.csv (10001 rows)
Successfully downloaded ratios_1994b.csv (2645 rows)
Successfully downloaded ratios_1995.csv (10001 rows)
Successfully downloaded ratios_1995b.csv (2004 rows)
Successfully downloaded ratios_1996.csv (10001 rows)
Successfully downloaded ratios_1996b.csv (1481 rows)
Successfully downloaded ratios_1997.csv (10001 rows)
Successfully downloaded ratios_1997b.csv (947 rows)
Successfully downloaded ratios_1998.csv (10001 

In [12]:
# Download Interest Expense Data
import requests
import os
import time
import csv
import io

def download_fdic_reports(start_year=1989, end_year=2023, output_dir="fdic_reports"):
    """
    Download FDIC financial reports (interest expense data) for year-end dates from start_year to end_year.
    Handles pagination for years with more than 10,000 records.
    
    Args:
        start_year (int): First year to download (inclusive)
        end_year (int): Last year to download (inclusive)
        output_dir (str): Directory to save downloaded files
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Base URL for the FDIC API
    base_url = "https://banks.data.fdic.gov/api/financials"
    
    # Full list of fields from the provided URL - these are interest expense related fields
    fields = [
        "CERT", "RSSDHCR", "NAMEFULL", "CITY", "STALP", "ZIP", "REPDTE", "BKCLASS", 
        "NAMEHCR", "OFFDOM", "SPECGRP", "SUBCHAPS", "ESTYMD", "INSDATE", "EFFDATE", 
        "MUTUAL", "PARCERT", "TRUST", "REGAGNT", "INSAGNT1", "FDICDBS", "FDICSUPV", 
        "FLDOFF", "FED", "OCCDIST", "OTSREGNM", "OFFOA", "CB", "EINTEXP", "EDEPDOM", 
        "EDEPFOR", "EFREPP", "EOTHINT", "NIM", "ELNATR"
    ]
    
    # Define limits and offsets for pagination
    limit = 10000
    offsets = [0, 10000]  # First batch: offset 0, second batch: offset 10000
    
    # Use year-end dates (December 31) for each year
    for year in range(start_year, end_year + 1):
        # Format the date as YYYYMMDD (December 31st)
        report_date = f"{year}1231"
        
        # Construct the base filter
        filters = f"ACTIVE:1 AND !(BKCLASS:NC) AND REPDTE:{report_date}"
        
        # Download data with pagination
        for i, offset in enumerate(offsets):
            # Create appropriate suffix for the filename - changed to use "intexp" prefix
            suffix = "" if i == 0 else "b"
            filename = f"intexp_{year}{suffix}.csv"
            file_path = os.path.join(output_dir, filename)
            
            print(f"Downloading FDIC interest expense report for year {year} (batch {i+1})...")
            
            # Construct parameters with the current offset
            params = {
                "filters": filters,
                "fields": ",".join(fields),
                "sort_by": "REPDTE",
                "sort_order": "DESC",
                "limit": str(limit),
                "offset": str(offset),
                "format": "csv",
                "download": "true",
                "filename": f"intexp_{year}{suffix}"
            }
            
            try:
                # Send the request
                response = requests.get(base_url, params=params)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Check the content before saving
                    content = response.content
                    content_str = content.decode('utf-8', errors='replace')
                    
                    # Check if we have valid CSV data with at least a header row
                    csv_reader = csv.reader(io.StringIO(content_str))
                    rows = list(csv_reader)
                    num_rows = len(rows)
                    
                    if num_rows <= 1:
                        print(f"Warning: Downloaded file for {year} (batch {i+1}) only has {num_rows} rows")
                        
                        # If it's the second batch and there's only a header or no data, it means there are no more records
                        if i > 0 and num_rows <= 1:
                            print(f"No more records for year {year} after offset {offset}. Skipping this file.")
                            continue
                        
                        # If there's an error message in the response, print it
                        if "error" in content_str.lower() or "invalid" in content_str.lower():
                            print(f"Error in response: {content_str[:200]}...")
                            continue
                    
                    # Save the content to a file
                    with open(file_path, 'wb') as f:
                        f.write(content)
                    
                    print(f"Successfully downloaded {filename} ({num_rows} rows)")
                    
                    # If there are fewer than limit records in the first batch, we don't need the second batch
                    if i == 0 and num_rows < limit + 1:  # +1 for header row
                        print(f"Year {year} has fewer than {limit} records. Skipping second batch.")
                        break
                        
                else:
                    print(f"Failed to download report for {year} (batch {i+1}). Status code: {response.status_code}")
                    if hasattr(response, 'text'):
                        print(f"Error details: {response.text[:200]}...")
            
            except Exception as e:
                print(f"Error downloading report for {year} (batch {i+1}): {str(e)}")
                
            # Add a delay between requests to avoid overwhelming the server
            time.sleep(3)  # Increased delay to 3 seconds to be more respectful to the server
    
    print(f"\nDownload complete. Files saved to '{output_dir}' directory.")

if __name__ == "__main__":
    # Execute the download function for years 1989-2023
    download_fdic_reports(1989, 2023, "C:/temp/fdic_reports")

Successfully downloaded intexp_1989.csv (10001 rows)
Successfully downloaded intexp_1989b.csv (6109 rows)
Successfully downloaded intexp_1990.csv (10001 rows)
Successfully downloaded intexp_1990b.csv (5375 rows)
Successfully downloaded intexp_1991.csv (10001 rows)
Successfully downloaded intexp_1991b.csv (4613 rows)
Successfully downloaded intexp_1992.csv (10001 rows)
Successfully downloaded intexp_1992b.csv (3974 rows)
Successfully downloaded intexp_1993.csv (10001 rows)
Successfully downloaded intexp_1993b.csv (3326 rows)
Successfully downloaded intexp_1994.csv (10001 rows)
Successfully downloaded intexp_1994b.csv (2645 rows)
Successfully downloaded intexp_1995.csv (10001 rows)
Successfully downloaded intexp_1995b.csv (2004 rows)
Successfully downloaded intexp_1996.csv (10001 rows)
Successfully downloaded intexp_1996b.csv (1481 rows)
Successfully downloaded intexp_1997.csv (10001 rows)
Successfully downloaded intexp_1997b.csv (947 rows)
Successfully downloaded intexp_1998.csv (10001 

In [14]:
# Download Demographic Data
import requests
import os
import time
import csv
import io

def download_fdic_reports(start_year=1989, end_year=2023, output_dir="fdic_reports"):
    """
    Download FDIC financial reports (demographic data) for year-end dates from start_year to end_year.
    Handles pagination for years with more than 10,000 records.
    
    Args:
        start_year (int): First year to download (inclusive)
        end_year (int): Last year to download (inclusive)
        output_dir (str): Directory to save downloaded files
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Base URL for the FDIC API
    base_url = "https://banks.data.fdic.gov/api/financials"
    
    # Full list of fields from the provided URL - these are demographic fields
    fields = [
        "ACTIVE", "BKCLASS", "CALLFORM", "CB", "CBSA", "CERT", "CITY", "EFFDATE", 
        "ESTYMD", "FDICDBS", "FDICDBSDESC", "FDICSUPV", "FDICSUPVDESC", "FED", 
        "FEDDESC", "FLDOFF", "INSAGNT1", "INSDATE", "MUTUAL", "NAME", "NAMEFULL", 
        "NAMEHCR", "OCCDIST", "OCCDISTDESC", "OFFDOM", "OFFFOR", "OFFOA", "OTSREGNM", 
        "PARCERT", "QBPRCOMLDESC", "REGAGNT", "REPDTE", "RSSDHCR", "RSSDID", "SPECGRP", 
        "SPECGRPDESC", "STALP", "STCNTY", "SUBCHAPS", "TRUST", "ZIP"
    ]
    
    # Define limits and offsets for pagination
    limit = 10000
    offsets = [0, 10000]  # First batch: offset 0, second batch: offset 10000
    
    # Use year-end dates (December 31) for each year
    for year in range(start_year, end_year + 1):
        # Format the date as YYYYMMDD (December 31st)
        report_date = f"{year}1231"
        
        # Construct the base filter
        filters = f"ACTIVE:1 AND !(BKCLASS:NC) AND REPDTE:{report_date}"
        
        # Download data with pagination
        for i, offset in enumerate(offsets):
            # Create appropriate suffix for the filename - changed to use "demographics" prefix
            suffix = "" if i == 0 else "b"
            filename = f"demographics_{year}{suffix}.csv"
            file_path = os.path.join(output_dir, filename)
            
            print(f"Downloading FDIC demographics report for year {year} (batch {i+1})...")
            
            # Construct parameters with the current offset
            params = {
                "filters": filters,
                "fields": ",".join(fields),
                "sort_by": "REPDTE",
                "sort_order": "DESC",
                "limit": str(limit),
                "offset": str(offset),
                "format": "csv",
                "download": "true",
                "filename": f"demographics_{year}{suffix}"
            }
            
            try:
                # Send the request
                response = requests.get(base_url, params=params)
                
                # Check if the request was successful
                if response.status_code == 200:
                    # Check the content before saving
                    content = response.content
                    content_str = content.decode('utf-8', errors='replace')
                    
                    # Check if we have valid CSV data with at least a header row
                    csv_reader = csv.reader(io.StringIO(content_str))
                    rows = list(csv_reader)
                    num_rows = len(rows)
                    
                    if num_rows <= 1:
                        print(f"Warning: Downloaded file for {year} (batch {i+1}) only has {num_rows} rows")
                        
                        # If it's the second batch and there's only a header or no data, it means there are no more records
                        if i > 0 and num_rows <= 1:
                            print(f"No more records for year {year} after offset {offset}. Skipping this file.")
                            continue
                        
                        # If there's an error message in the response, print it
                        if "error" in content_str.lower() or "invalid" in content_str.lower():
                            print(f"Error in response: {content_str[:200]}...")
                            continue
                    
                    # Save the content to a file
                    with open(file_path, 'wb') as f:
                        f.write(content)
                    
                    print(f"Successfully downloaded {filename} ({num_rows} rows)")
                    
                    # If there are fewer than limit records in the first batch, we don't need the second batch
                    if i == 0 and num_rows < limit + 1:  # +1 for header row
                        print(f"Year {year} has fewer than {limit} records. Skipping second batch.")
                        break
                        
                else:
                    print(f"Failed to download report for {year} (batch {i+1}). Status code: {response.status_code}")
                    if hasattr(response, 'text'):
                        print(f"Error details: {response.text[:200]}...")
            
            except Exception as e:
                print(f"Error downloading report for {year} (batch {i+1}): {str(e)}")
                
            # Add a delay between requests to avoid overwhelming the server
            time.sleep(3)  # Increased delay to 3 seconds to be more respectful to the server
    
    print(f"\nDownload complete. Files saved to '{output_dir}' directory.")

if __name__ == "__main__":
    # Execute the download function for years 1989-2023
    download_fdic_reports(1989, 2023, "C:/temp/fdic_reports")

Successfully downloaded demographics_1989.csv (10001 rows)
Successfully downloaded demographics_1989b.csv (6109 rows)
Successfully downloaded demographics_1990.csv (10001 rows)
Successfully downloaded demographics_1990b.csv (5375 rows)
Successfully downloaded demographics_1991.csv (10001 rows)
Successfully downloaded demographics_1991b.csv (4613 rows)
Successfully downloaded demographics_1992.csv (10001 rows)
Successfully downloaded demographics_1992b.csv (3974 rows)
Successfully downloaded demographics_1993.csv (10001 rows)
Successfully downloaded demographics_1993b.csv (3326 rows)
Successfully downloaded demographics_1994.csv (10001 rows)
Successfully downloaded demographics_1994b.csv (2645 rows)
Successfully downloaded demographics_1995.csv (10001 rows)
Successfully downloaded demographics_1995b.csv (2004 rows)
Successfully downloaded demographics_1996.csv (10001 rows)
Successfully downloaded demographics_1996b.csv (1481 rows)
Successfully downloaded demographics_1997.csv (10001 row